In [6]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, load
import comet_ml
import os
from deta import Deta
from dotenv import load_dotenv

load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))
comet_ml_api_key = os.environ.get("COMET_ML_API_KEY")
api = comet_ml.api.API(comet_ml_api_key)


In [20]:

# api.download_registry_model("drdevinhopkins", "verticaltbs",
#                             version="1.0.0", output_path="comet-ml-models", expand=True, stage=None)

# loaded_model = load("comet-ml-models/verticalTBS.np")
loaded_model = load("models/verticalTBS.np")

# set_log_level("ERROR")


In [21]:

target_column = 'Total Vertical TBS'
df = pd.read_csv(
    'data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')
# df['ID'] = 'test'


In [26]:
df.tail(5).columns

Index(['Stretcher Pts hrly', 'Stretcher Pts cum', 'Ambulatory Pts hrly',
       'Ambulatory Pts cum', 'Total Inflow hrly', 'Total Inflow cum',
       'Ambulances hrly', 'Ambulances cum', 'FLS hrly', 'Adm requests cum',
       'Admissions cum', 'Pts waiting for admission CUM',
       'Total Stretcher pts', 'Triage hallway pts', 'Triage hallway pts TBS',
       'Re-Oriented Nurse cum', 'Re-Oriented MD QTrack D/C',
       'Re-Oriented MD QTrack NotD/C', 'Resus Pts',
       'Totalpts in PODs except Psych', 'Green Pts', 'Green Pts TBS',
       'Yellow PTS', 'Yellow Pts TBS', 'Orancge Pts except psych',
       'Orange Pts TBS', 'Consults > 2h in PODS except IM',
       'Consult for IM >4h in PODS', 'Plain films reqs > 2 h in PODs',
       'CTs reqs > 2 h in PODs', 'Post POD (Family room)',
       'Stretcher Pts in Vertical', 'Stretcher Pts TBS in Vertical',
       'Stretcher Pts in Vertical on Lazyboy',
       'Vertical Pts Waiting for Results', 'Ambulatory Pts in Vertical',
       'Ambulato

In [23]:

# periods=m.n_forecasts, n_historic_predictions=False
future = loaded_model.make_future_dataframe(df, periods=12)
future

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y,Stretcher Overflow
0,1,13,1,5,2,18,0,4,0,7,...,2,1,0,1,15,15,2023-06-09 06:00:00,1,3,0
1,7,20,5,10,12,30,4,8,0,7,...,2,1,0,1,15,15,2023-06-09 07:00:00,4,9,0
2,4,24,7,17,11,41,0,8,0,7,...,2,1,1,1,15,15,2023-06-09 08:00:00,2,15,0
3,5,29,8,25,13,54,3,11,0,7,...,7,1,0,4,15,15,2023-06-09 09:00:00,1,12,5
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 10:00:00,None,None,None
5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 11:00:00,None,None,None
6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 12:00:00,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 13:00:00,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 14:00:00,None,None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 15:00:00,None,None,None


In [24]:

# from pytorch_lightning import loggers
# from pytorch_lightning.utilities import rank_zero_only

# class TBLogger(loggers.TensorBoardLogger):
#     @rank_zero_only
#     def log_metrics(self, metrics, step):
#         metrics.pop('epoch', None)
#         return super().log_metrics(metrics, step)

forecast = loaded_model.predict(future,
                                # raw=False,
                                # decompose=False
                                )


Predicting DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x144 and 188x36)

In [ ]:

forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('forecasts/verticalTBS_raw.csv')
start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='H'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'verticalTBS': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('forecasts/verticalTBS.csv', index=False)

forecasts = deta.Drive("forecasts")

forecasts.put('verticalTBS.csv', path='forecasts/verticalTBS.csv')


In [27]:
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save
import pickle
import os
from dotenv import load_dotenv

load_dotenv()

experiment = Experiment(
    api_key=os.environ.get('COMET_ML_API_KEY'),
    project_name="verticalTBS",
    workspace="drdevinhopkins",
)
api = API(os.environ.get('COMET_ML_API_KEY'))

set_log_level("ERROR")

target_column = 'Total Vertical TBS'

df = pd.read_csv(
    'data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

params = {
    # growth='off',
    'yearly_seasonality': False,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 4,
    'n_forecasts': 12,
    'changepoints_range': 0.95,
    'n_changepoints': 50,
    'quantiles': [0.2, 0.5, 0.8]
    # num_hidden_layers=4,
    # d_hidden=36,
    # learning_rate=0.005,
}

m = NeuralProphet(**params)
m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")
metrics = m.fit(df,
                freq='H',
                # progress='plot'
                )
print(metrics.tail(1))

experiment.log_parameters(params)
experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())

save(m, "models/verticalTBS.np")

experiment.log_model("verticalTBS", "models/verticalTBS.np")


# with open('models/verticalTBS_forecast_model.pkl', "wb") as f:
#     pickle.dump(m, f)

experiment_name = experiment.name

experiment.end()


api.delete_registry_model('drdevinhopkins', 'verticalTBS')

apiexp = api.get('drdevinhopkins/verticalTBS/'+experiment_name)

apiexp.register_model('verticalTBS')


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/verticaltbs/5cb5e480adaa4b8b82a0df21d8910482

Finding best initial lr: 100%|██████████| 262/262 [00:02<00:00, 115.49it/s]


Epoch 78: 100%|██████████| 78/78 [00:04<00:00, 16.47it/s, loss=0.0253, v_num=1, MAE=4.140, RMSE=5.390, Loss=0.0244, RegLoss=0.000]   
        MAE     RMSE     Loss  RegLoss  epoch
77  4.13928  5.39057  0.02435      0.0     77


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/verticaltbs/5cb5e480adaa4b8b82a0df21d8910482
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss [79]  : (0.024350466206669807, 3.0502946376800537)
COMET INFO:     MAE [79]   : (4.139280319213867, 55.25100326538086)
COMET INFO:     RMSE [79]  : (5.390570163726807, 70.98001098632812)
COMET INFO:     RegLoss    : 0.0
COMET INFO:     epoch [79] : (0, 77)
COMET INFO:     hp_metric  : -1
COMET INFO:   Parameters:
COMET INFO:     changepoints_range : 0.95
COMET INFO:     daily_seasonality  : True
COMET INFO:     n_changepoints     : 50
COMET INFO:     n_forecasts        : 12
COMET INFO:     n_lags            

In [30]:
future = loaded_model.make_future_dataframe(df, periods=12, n_historic_predictions=True)
future.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y,Stretcher Overflow
29848,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 18:00:00,None,None,None
29849,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 19:00:00,None,None,None
29850,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 20:00:00,None,None,None
29851,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 21:00:00,None,None,None
29852,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 22:00:00,None,None,None


In [31]:
forecast = loaded_model.predict(future)

Predicting: 0it [01:36, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1024x144 and 188x36)

In [2]:
%%sh
git clone https://github.com/ourownstory/neural_prophet.git
cd neural_prophet
pip install .

Cloning into 'neural_prophet'...


Processing /Users/devinhopkins/Documents/GitHub/hourly-report-neuralprophet/neural_prophet
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached kaleido-0.2.1-py2.py3-none-macosx_10_11_x86_64.whl (85.2 MB)
  Using cached captum-0.6.0-py3-none-any.whl (1.3 MB)
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached plotly-5.15.0-py2.py3-none-any.whl (15.5 MB)
  Using cached numpy-1.23.5-cp310-cp310-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached torch-1.13.1-cp310-none-macosx_10_9_x86_64.whl (135.3 MB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
  Using cached tensorboard-2

In [1]:
!pip install neuralprophet==0.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached numpy-1.23.5-cp310-cp310-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached pandas-2.0.2-cp310-cp310-macosx_10_9_x86_64.whl (11.8 MB)
  Using cached captum-0.6.0-py3-none-any.whl (1.3 MB)
  Using cached plotly_resampler-0.8.3.2.tar.gz (46 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached matplotlib-3.7.1-cp310-cp310-macosx_10_12_x86_64.whl (7.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.4 MB/s eta 0:00:0000:0100:01
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.5 MB/s eta 0:00:0000:0100:01
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl 

In [ ]:
import pandas as pd
from neuralprophet import NeuralProphet

df = pd.read_csv(
    'data/since-2020.csv')
df.ds = pd.to_datetime(df.

In [3]:
import pandas as pd
from neuralprophet import NeuralProphet

target_column = 'Total Vertical TBS'

df = pd.read_csv(
    'data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

params = {
    # growth='off',
    'yearly_seasonality': False,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 4,
    'n_forecasts': 12,
    'changepoints_range': 0.95,
    'n_changepoints': 50,
    'quantiles': [0.2, 0.5, 0.8]
    # num_hidden_layers=4,
    # d_hidden=36,
    # learning_rate=0.005,
}

m = NeuralProphet(**params)
m = m.add_lagged_regressor(names=regressors)
# m = m.add_country_holidays("CA")
metrics = m.fit(df, #df[['ds', 'y']],
                freq='H',
                # progress='plot'
                )
print(metrics.tail(1))

INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', number of lags for regressor is set to Autoregression number of lags (4)
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.256% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.forecaster.__handle_missing_data) - 289 missing dates added.
WARNING - (NP.forecaster.__handle_missing_data) - 289 missing values in column y were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 289 NaN values in column y were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 289 missing values in column Stretcher Pts hrly were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 289 NaN values in column Stretcher Pts hrly were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - 289 

Finding best initial lr:   0%|          | 0/262 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

         MAE      RMSE      Loss  RegLoss  epoch
77  4.183859  5.428222  0.024842      0.0     77


In [4]:
# future = m.make_future_dataframe(df[['ds','y']], periods=12)
future = m.make_future_dataframe(df, periods=12)
future

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.256% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y,Stretcher Overflow
0,7,20,5,10,12,30,4,8,0,7,...,2,1,0,1,15,15,2023-06-09 07:00:00,4,9,0
1,5,25,6,16,11,41,0,8,0,7,...,2,1,1,1,15,15,2023-06-09 08:00:00,2,15,0
2,8,33,5,21,13,54,3,11,0,7,...,7,1,0,4,15,15,2023-06-09 09:00:00,1,12,5
3,7,40,7,28,14,68,3,14,0,7,...,5,0,0,2,16,15,2023-06-09 10:00:00,3,12,5
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 11:00:00,None,None,None
5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 12:00:00,None,None,None
6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 13:00:00,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 14:00:00,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 15:00:00,None,None,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,2023-06-09 16:00:00,None,None,None


In [20]:
df.tail()

,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,y,Stretcher Overflow
29836,1,13,1,5,2,18,0,4,0,7,...,2,1,0,1,15,15,2023-06-09 06:00:00,1,3,0
29837,7,20,5,10,12,30,4,8,0,7,...,2,1,0,1,15,15,2023-06-09 07:00:00,4,9,0
29838,5,25,6,16,11,41,0,8,0,7,...,2,1,1,1,15,15,2023-06-09 08:00:00,2,15,0
29839,8,33,5,21,13,54,3,11,0,7,...,7,1,0,4,15,15,2023-06-09 09:00:00,1,12,5
29840,7,40,7,28,14,68,3,14,0,7,...,5,0,0,2,16,15,2023-06-09 10:00:00,3,12,5


In [5]:
forecast = m.predict(future)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 93.75% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 93.75% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 471it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/neuralprophet-old/lib/python3.10/site-packages/neuralprophet/forecaster.py:3281: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/neuralprophet-old/lib/python3.10/site-packages/neuralprophet/forecaster.py:3281: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/devinhopkins/anaconda3/envs/neural

In [6]:
forecast

,ds,y,yhat1,yhat2,yhat3,yhat4,yhat5,yhat6,yhat7,yhat8,...,lagged_regressor_Stretcher Overflow6,lagged_regressor_Stretcher Overflow7,lagged_regressor_Stretcher Overflow8,lagged_regressor_Stretcher Overflow9,lagged_regressor_Stretcher Overflow10,lagged_regressor_Stretcher Overflow11,lagged_regressor_Stretcher Overflow12,trend,season_weekly,season_daily
0,2023-06-09 07:00:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-06-09 08:00:00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-06-09 09:00:00,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-06-09 10:00:00,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-06-09 11:00:00,NaN,15.00806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.142651,-0.128214,2.518451
5,2023-06-09 12:00:00,NaN,NaN,18.279747,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.142099,-0.047911,4.550396
6,2023-06-09 13:00:00,NaN,NaN,NaN,20.879223,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.141551,0.031893,4.978218
7,2023-06-09 14:00:00,NaN,NaN,NaN,NaN,23.087626,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.141000,0.110158,4.993111
8,2023-06-09 15:00:00,NaN,NaN,NaN,NaN,NaN,24.500097,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.140452,0.185843,6.007442
9,2023-06-09 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,26.466923,NaN,NaN,...,-1.708809,NaN,NaN,NaN,NaN,NaN,NaN,6.139900,0.257918,6.154595


In [7]:
forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('forecasts/verticalTBS_raw.csv')
start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='H'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'verticalTBS': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('forecasts/verticalTBS.csv', index=False)

In [8]:
forecast_output

,ds,verticalTBS,timestamp
0,2023-06-09 11:00:00,15.008060,2023-06-09 12:48:34
1,2023-06-09 12:00:00,18.279747,2023-06-09 12:48:34
2,2023-06-09 13:00:00,20.879223,2023-06-09 12:48:34
3,2023-06-09 14:00:00,23.087626,2023-06-09 12:48:34
4,2023-06-09 15:00:00,24.500097,2023-06-09 12:48:34
5,2023-06-09 16:00:00,26.466923,2023-06-09 12:48:34
6,2023-06-09 17:00:00,24.233524,2023-06-09 12:48:34
7,2023-06-09 18:00:00,21.033514,2023-06-09 12:48:34
8,2023-06-09 19:00:00,20.452448,2023-06-09 12:48:34
9,2023-06-09 20:00:00,19.873341,2023-06-09 12:48:34
